## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-14-23-38-21 +0000,nyt,Trump Adds 17% Tariff on Tomatoes From Mexico,https://www.nytimes.com/2025/07/14/business/ec...
1,2025-07-14-23-20-42 +0000,bbc,India can't wish away coal - but can it be mad...,https://www.bbc.com/news/articles/cpd184g6dj8o
2,2025-07-14-23-17-40 +0000,bbc,Beyoncé's unreleased music stolen from car dur...,https://www.bbc.com/news/articles/cdx5p4we9qjo
3,2025-07-14-23-06-25 +0000,nypost,Ground stops spurred by tropical storm causes ...,https://nypost.com/2025/07/14/us-news/ground-s...
4,2025-07-14-23-01-52 +0000,nypost,Board of federal judges blocks Trump-appointed...,https://nypost.com/2025/07/14/us-news/board-of...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2360/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,12
303,hit,4
373,public,4
226,new,4
5,india,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
42,2025-07-14-18-36-14 +0000,latimes,Trump officials vow to intensify immigration r...,https://www.latimes.com/california/story/2025-...,29
39,2025-07-14-19-10-57 +0000,latimes,Trump administration asks appeals court to lif...,https://www.latimes.com/california/story/2025-...,27
69,2025-07-14-01-15-53 +0000,bbc,How Trump public broadcasting cuts could hit r...,https://www.bbc.com/news/articles/c20w51rkw4no,25
20,2025-07-14-21-34-30 +0000,bbc,Trump threatens Russia with tariffs while unve...,https://www.bbc.com/news/articles/czdv20v9lp1o,24
4,2025-07-14-23-01-52 +0000,nypost,Board of federal judges blocks Trump-appointed...,https://nypost.com/2025/07/14/us-news/board-of...,24


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
42,29,2025-07-14-18-36-14 +0000,latimes,Trump officials vow to intensify immigration r...,https://www.latimes.com/california/story/2025-...
35,21,2025-07-14-20-26-48 +0000,nypost,Colo. Dem DA ‘will be recalled’ for moving to ...,https://nypost.com/2025/07/14/us-news/colo-da-...
9,21,2025-07-14-22-29-19 +0000,nypost,Founder of Cali pot farm where ICE rescued doz...,https://nypost.com/2025/07/14/us-news/founder-...
32,21,2025-07-14-20-41-51 +0000,nypost,NY Gov. Hochul builds $17.5M campaign war ches...,https://nypost.com/2025/07/14/us-news/ny-gov-h...
18,20,2025-07-14-21-43-35 +0000,nypost,Texas flood volunteers ordered to evacuate as ...,https://nypost.com/2025/07/14/us-news/texas-fl...
11,18,2025-07-14-22-20-02 +0000,nypost,NYC beatdown victim turns tables and opens fir...,https://nypost.com/2025/07/14/us-news/nyc-beat...
33,17,2025-07-14-20-37-28 +0000,nyt,The Economy Has Been Resilient. The New Round ...,https://www.nytimes.com/2025/07/14/us/politics...
23,17,2025-07-14-21-27-46 +0000,nypost,DOJ defends bringing case against Ghislaine Ma...,https://nypost.com/2025/07/14/us-news/doj-defe...
54,17,2025-07-14-14-55-07 +0000,bbc,"Boeing fuel switches safe, regulator says afte...",https://www.bbc.com/news/articles/ce9xpgnx3vdo
8,16,2025-07-14-22-34-06 +0000,nypost,Anti-Zohran Mamdani ‘move to Ohio’ billboard i...,https://nypost.com/2025/07/14/us-news/anti-zoh...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
